In [ ]:
from IPython.display import HTML
HTML('''<script>code_show=true;function code_toggle() {if (code_show){$('div.input').hide();} else {$('div.input').show();}code_show = !code_show}$( document ).ready(code_toggle);</script><form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


# pyDAMPF - Motivus

Welcome to pyDAMPF: a Python package for modeling mechanical properties of hygroscopic materials under interaction with a nanoprobe. You can find out more on the [official repo](https://github.com/govarguz/pyDAMPF).

Processing capabilitites are provided by [Motivus](https://motivus.cl/documentation). Code adaptation is availabe [here](https://github.com/m0tivus/mypmmoda)


## How to use this notebook



2.   Run all cells
2.   You should see a graphical user interface with several options as tabs
3.   Select which cantilevers you want to analyze
4.   Adjust variables as you need. 
5.   Go to execute and click the button (this process can take a few minutes). It is recommended that the number of threads is equal to the cantilevers.
6.   After all the cases are evaluated click on "the simulation is completed" button.
7.   Choose between graphical or quantitative summary of the execution (graphical summary may take some seconds. Phase and Amplitude related data is not available)

# Previous results

Use the last cell on the notebook to analyze previous simulations results. The results are listed based on simulation completion.

> IMPORTANT: If you lose connection to the instance you can not monitor progress trough the progress bar.



In [ ]:
%%capture
#@title
%cd ~
! git clone https://github.com/govarguz/pyDAMPF.git pyDAMPF2.1
%cd pyDAMPF2.1
! git checkout motivus && git pull

! rm -rf EXECUTE_pyDAMPF/PARALLELBASIC_* EXECUTE_pyDAMPF/PLOTS

! pip install -r requirements.txt

In [ ]:
#@title
import re
import os
from IPython.display import display
import ipywidgets as widgets
import threading
import time
import asyncio

class ResultConsolidator:
  def __init__(self):
    self.setup_dir()

  def setup_dir(self):
    current_time = time.localtime()
    name = time.strftime('%Y-%m-%dT%H:%M:%S', current_time)
    self.dir = os.path.join("/", "home", "jovyan", "executions", name)

  def copy_results(self, src):
    shutil.copytree(src, self.dir)
    

class ProgressWidget:
  def __init__(self, directory):
    self.max = self.get_max()
    self.value = 0
    self.widget = self.build_widget()
    self.directory = directory
    self.stop_button = widgets.Button(
        description='Cancel',
        disabled=False,
        button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Stop current processing'
    )
    
    self.is_cancelled = False

    #self.loop = asyncio.new_event_loop()
    self.thread = threading.Thread(target=self.check_until_completed)
    self.thread.start()

    self.stop_button.on_click(self.cancel_task)

    display(self.widget, self.stop_button)
    #self.start_periodic_refresh()

  def cancel_task(self, *_):
    self.is_cancelled = True
    #self.loop.call_soon_threadsafe(self.loop.stop)

  def build_widget(self):
    return widgets.IntProgress(
        value=self.value,
        min=0,
        max=self.max,
        description=f"Processing {self.max} tasks ...",
        bar_style='info',
        style={'bar_color': '#cc5de7'},
        orientation='horizontal'
    )
  
  def update(self):
    self.max = self.get_max()
    self.widget.max = self.max

    self.update_value()
    self.widget.value = self.value

  def did_complete(self):
    return self.max != 0 and self.max == self.value

  def get_max(self):
    return self.get_total_case_count()

  def update_value(self):
    self.value = self.get_solved_case_count()

  def get_total_case_count(self):    
    my_path = os.getcwd()+ "/EXECUTE_pyDAMPF"
    matches = list()
    r = re.compile("case\d+\.in")
    for _root, _dirs, files in os.walk(my_path, topdown=False):
      for name in files:
        matches.append(name) if r.match(name) else None
    return len(matches)

  def get_solved_case_count(self):
    my_path = os.getcwd()+ "/EXECUTE_pyDAMPF"
    matches = list()
    r = re.compile("zcdom\d+\.dfo")
    for _root, _dirs, files in os.walk(my_path, topdown=False):
      for name in files:
        matches.append(name) if r.match(name) else None
    return len(matches)
  
  def check_until_completed(self):
    while not self.did_complete():
      if self.is_cancelled:
        break
      self.update()
      time.sleep(1)
    self.on_finished()
    
  def on_finished(self):
    if self.is_cancelled:
        self.widget.description = "Cancelled"
    else:
        self.widget.description = "Finished!"
        ResultConsolidator().copy_results(self.directory)
        
    
  def start_periodic_refresh(self):
        pass
    #asyncio.run(self.check_until_completed())




In [ ]:
#@title

#=======================pyDAMPF==============================
#The code written below is part of the pyDAMPF project and has
#been developed as a joint project between the IJS and UMSA. This 
#software will enable students and researchers from the 
#AFM community to easily plan AFM experiments.
#======================CREDITS==============================
#
# PI Horacio V. Guzman Ph.D.
# B.Sc. Willy Menacho N.
#
#
#We thank you in advance for sending your feedback and/or 
#suggestions to:
#             horacio.v.g@gmail.com
#
#======================pyDAMPF LICENSE==============================

#Copyright (C) 2022  Horacio V. Guzman and Willy Menacho N.

#This program is free software: you can redistribute it and/or modify
#it under the terms of the GNU General Public License as published by
#the Free Software Foundation, either version 3 of the License, or
#(at your option) any later version.

#This program is distributed in the hope that it will be useful,
#but WITHOUT ANY WARRANTY; without even the implied warranty of
#MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#GNU General Public License for more details.

#You should have received a copy of the GNU General Public License
#along with this program.  If not, see <http://www.gnu.org/licenses/>.
#
#====================================================================

import ipywidgets as widgets
import numpy as np
import os
import shutil
from tempfile import mkstemp
import re
import pandas as pd
#=============TO HIDE THE CODE===========

#=============FUNTIONS==================
UBI=os.getcwd()

def permutador(lst, data):
    y, x = np.shape(data)
    arr_aux = np.array([0])
    lst_arr = []
    for i in range(y):
        arr_aux = data[i]
        arr_tmp = np.insert(arr_aux, x, lst[0])
        arr_family = arr_tmp
        for j in lst[1:]:
            arr_tmp = np.insert(arr_aux, x, j)
            arr_family = np.concatenate((arr_family, arr_tmp))
        arr_family = np.reshape(arr_family, (len(lst), x+1))
        lst_arr.append(arr_family)
    final_arr = np.concatenate(lst_arr)
    y, x = np.shape(final_arr)
    final_arr[:,0] = np.arange(1,y+1)
    return final_arr

def gran_permutador(lst, data2):
    aux = permutador(lst[0], data2)
    for i in lst[1:]:
        aux = permutador(i, aux)
    y, x = np.shape(aux)
    aux[:,0] = np.arange(1,y+1)
    return aux

def gen_limites(tcases, factor):
    paralleliz = int(tcases/factor)
    aux = tcases - (paralleliz*factor) #casos sobrantes
    lim_inf = 0
    lim_sup = lim_inf + (paralleliz)
    if aux>0:
        lim_sup = lim_sup + 1
        aux = aux -1
    lst = [(lim_inf, lim_sup)]
    for i in range(1,factor):
        lim_inf = lim_sup
        lim_sup = lim_inf + (paralleliz)
        if aux>0:
            lim_sup = lim_sup + 1
            aux = aux - 1
        lst.append((lim_inf, lim_sup))
    return lst

def reemplazo(source_file_path, pattern, substring):
    fh, target_file_path = mkstemp()
    with open(target_file_path, 'w') as target_file:
        with open(source_file_path, 'r') as source_file:
            for line in source_file:
                target_file.write(line.replace(pattern, substring))
    #print(source_file_path)
    os.remove(source_file_path)
    shutil.move(target_file_path, source_file_path)
    
def change_dir():
    my_path = os.getcwd()
    ls = os.listdir(my_path)
    if "PARALLELBASIC_0" in ls:
        r = re.compile("PARALLELBASIC_[0-9]")
        lst = list(filter(r.match, ls))
        nums = []
        lst.sort(reverse = True)
        for i in lst:
            aux = i.split("_")[1]
            nums.append(int(aux))
            new_dir = "PARALLELBASIC_" + str(int(aux) + 1)
            os.rename(i, new_dir)
        os.makedirs("PARALLELBASIC_0")
    else:
        os.makedirs("PARALLELBASIC_0")
        

        
def inputs_processor (x=False):
    if x:
        #==================TEMPALL.TXT=====================
        #### values for the numerical integrator
        nper1 = [nper.value]
        npp1 = [npp.value]
        nperfin1 = [nperfin.value]
        zcmin1 = [zcmin.value]
        dzc1 = [dzc.value]
        fixzc1 = [fixzc.value]

        #### essential values 
        a01 = [a0.value]
        Es1 = [Es.value]
        q1 = [q.value]
        RH1 = [RH.value]
        
        #### values for interactive forces with select check box
        if left_box2.children[1].value==True:
            ham1=[ham.value]
        else:
            ham1=[0]
        epsilon11 = [epsilon1.value]
        
        if left_box2.children[2].value==True:
            eps1 = [eps.value]
            eps01 = [eps0.value] 
            sigmat1 = [sigmat.value] 
            sigmas1 = [sigmas.value]
            debye1 = [debye.value]
        else:
            #######IF THIS VALUES ARE 0 = INDETERMINATION ##### FOR FIX THIS:
            eps1=[1]
            eps01=[1]
            sigmat1=[0]
            sigmas1=[0]
            debye1=[1]
            
        if left_box2.children[3].value==True:
            eta1 = [eta.value]
        else:
            eta1=[0]
            
        if left_box2.children[4].value==True:
            epsilon21 = [epsilon2.value]
        else:
            epsilon21=[0]
            
        if left_box2.children[5].value==True:
            ljmin1 = [ljmin.value]
            lenght1 = [lenght.value]
        else:
            ljmin1=[0]
            lenght1=[0]
            
        #mtip = [float(input('Tip magn [A m2] ' ))]
        #msample = [float(input('Surface magn [A m2] ' ))]  

        dataaux = np.genfromtxt("cantilevers_data.txt")   
        mascara=[cant1.value,cant2.value,cant3.value,cant4.value,cant5.value,cant6.value,cant7.value,cant8.value,cant9.value,cant10.value
                 ,cant11.value,cant12.value,cant13.value,cant14.value,cant15.value,cant16.value,cant17.value,cant18.value,cant19.value,cant20.value
                 ,cant21.value,cant22.value,cant23.value,cant24.value,cant25.value,cant26.value,cant27.value,cant28.value,cant29.value,cant30.value
                 ,cant31.value,cant32.value,cant33.value,cant34.value,cant35.value,cant36.value,cant37.value,cant38.value,cant39.value,cant40.value
                 ,cant41.value,cant42.value,cant43.value,cant44.value,cant45.value,cant46.value,cant47.value,cant48.value,cant49.value,cant50.value
                 ,cant51.value,cant52.value,cant53.value,cant54.value,cant55.value,cant56.value,cant57.value,cant58.value,cant59.value,cant60.value
                 ,cant61.value,cant62.value,cant63.value,cant64.value,cant65.value,cant66.value,cant67.value,cant68.value,cant69.value,cant70.value
                 ,cant71.value,cant72.value,cant73.value,cant74.value,cant75.value,cant76.value,cant77.value,cant78.value,cant79.value,cant80.value
                 ,cant81.value,cant82.value,cant83.value,cant84.value,cant85.value,cant86.value,cant87.value,cant88.value,cant89.value,cant90.value
                 ,cant91.value,cant92.value,cant93.value,cant94.value,cant95.value,cant96.value,cant97.value,cant98.value,cant99.value,cant100.value
                 ,cant101.value,cant102.value,cant103.value,cant104.value,cant105.value,cant106.value,cant107.value,cant108.value,cant109.value,cant110.value
                 ,cant111.value,cant112.value,cant113.value,cant114.value,cant115.value,cant116.value,cant117.value,cant118.value,cant119.value,cant120.value
                 ,cant121.value,cant122.value,cant123.value,cant124.value,cant125.value,cant126.value,cant127.value,cant128.value,cant129.value,cant130.value
                 ,cant131.value,cant132.value,cant133.value,cant134.value,cant135.value,cant136.value,cant137.value,cant138.value,cant139.value,cant140.value
                 ,cant141.value,cant142.value,cant143.value,cant144.value,cant145.value,cant146.value,cant147.value,cant148.value,cant149.value,cant150.value
                 ,cant151.value,cant152.value,cant153.value,cant154.value,cant155.value,cant156.value,cant157.value,cant158.value,cant159.value,cant160.value
                 ,cant161.value,cant162.value,cant163.value,cant164.value,cant165.value,cant166.value,cant167.value,cant168.value,cant169.value,cant170.value
                 ,cant171.value,cant172.value,cant173.value,cant174.value,cant175.value,cant176.value,cant177.value,cant178.value,cant179.value,cant180.value
                 ,cant181.value,cant182.value,cant183.value,cant184.value,cant185.value,cant186.value,cant187.value,cant188.value,cant189.value,cant190.value
                 ,cant191.value,cant192.value,cant193.value,cant194.value,cant195.value,cant196.value,cant197.value,cant198.value,cant199.value,cant200.value
                 ,cant201.value,cant202.value,cant203.value,cant204.value,cant205.value,cant206.value,cant207.value,cant208.value,cant209.value,cant210.value
                 ,cant211.value,cant212.value,cant213.value,cant214.value,cant215.value,cant216.value,cant217.value,cant218.value,cant219.value,cant220.value
                 ,cant221.value,cant222.value,cant223.value,cant224.value,cant225.value,cant226.value,cant227.value,cant228.value,cant229.value,cant230.value
                 ,cant231.value,cant232.value,cant233.value,cant234.value,cant235.value,cant236.value,cant237.value,cant238.value,cant239.value,cant240.value
                 ,cant241.value,cant242.value]
        df = pd.DataFrame(data=dataaux)
        cant=df[mascara]
        data=cant.to_numpy()
        list_var = [nper1,npp1,nperfin1,zcmin1,dzc1,fixzc1,a01,Es1,q1,ham1,eta1,epsilon21,epsilon11,eps1,eps01,sigmat1,sigmas1,debye1,ljmin1,lenght1,RH1]

        a, b = np.shape(data)
        final = gran_permutador(list_var, data)
        f_name = "tempall.txt"
        np.savetxt(f_name, final)
        directory = os.getcwd()
        shutil.copy(directory+'/tempall.txt' , directory+'/EXECUTE_pyDAMPF/')
        shutil.copy(directory+'/tempall.txt' , directory+'/EXECUTE_pyDAMPF/pyDAMPF_BASE/nrun/')
        print('TEMPALL.TXT WAS CREATED')
        
        #============PARALLEL METHOD=============
        os.chdir(directory+'/EXECUTE_pyDAMPF')
        tempall = np.genfromtxt('tempall.txt')
        tcases = len(tempall[:,0])
        factor = int(hilos.value) 
        lst = gen_limites(tcases, factor)
        change_dir()
        for i in range (1,factor+1):
            lim_inferior, lim_superior = lst[i-1][0], lst[i-1][1]
            direc = os.getcwd()
            direc2 = direc+'/pyDAMPF_BASE/'
            direc3 = direc+'/PARALLELBASIC_0/'+str(i)+'/'
            shutil.copytree(direc2, direc3)
            factorantiguo='factor=1'
            factornuevo='factor='+str(factor)
            rangoantiguo='(0,paraleliz)'
            rangonuevo='('+str(lim_inferior)+","+str(lim_superior)+')'
            os.chdir(direc+'/PARALLELBASIC_0/'+str(i))
            pyname='nrun/generate_cases.py'
            newpath=direc+'/PARALLELBASIC_0/'+str(i)+'/'+pyname
            reemplazo(newpath, factorantiguo, factornuevo)
            reemplazo(newpath, rangoantiguo, rangonuevo)
            os.chdir(direc)
        print('PARALLELIZATION COMPLETE')
        
        #============JOB_COMPUTER FOR NOW========
        direc = os.getcwd()
        for i in range(1,factor+1):  #change
            os.chdir(direc+'/PARALLELBASIC_0/'+str(i)+'/nrun')
            os.system('python3 generate_cases.py &')
            os.chdir(direc)
        print('SIMULATION IN PROGRESS')
        os.chdir(UBI)

        widget = ProgressWidget(direc)

# New simulation

In [ ]:
#@title
####################################INTERFAZ###############################################

    
#=========POSITION FOR BOTOMS========
style1 = {'description_width': '50px'}
layout1 = {'width': '300px'}

#==================CANTILEVERS===================
cant1=widgets.Checkbox(value=True,description='Cant. [1]  Model [160AC-FG]',disabled=False, style=style1, layout=layout1)
cant2=widgets.Checkbox(value=True,description='Cant. [2]  Model [160AC-GG]',disabled=False, style=style1, layout=layout1)
cant3=widgets.Checkbox(value=True,description='Cant. [3]  Model [160AC-NA]',disabled=False, style=style1, layout=layout1)
cant4=widgets.Checkbox(value=True,description='Cant. [4]  Model [160AC-NG]',disabled=False, style=style1, layout=layout1)
cant5=widgets.Checkbox(value=True,description='Cant. [5]  Model [160AC-NN]',disabled=False, style=style1, layout=layout1)
cant6=widgets.Checkbox(value=True,description='Cant. [6]  Model [160AC-SG]',disabled=False, style=style1, layout=layout1)
cant7=widgets.Checkbox(value=True,description='Cant. [7]  Model [200AC-NA]',disabled=False, style=style1, layout=layout1)
cant8=widgets.Checkbox(value=True,description='Cant. [8]  Model [240AC-FG]',disabled=False, style=style1, layout=layout1)
cant9=widgets.Checkbox(value=True,description='Cant. [9]  Model [240AC-GG]',disabled=False, style=style1, layout=layout1)
cant10=widgets.Checkbox(value=True,description='Cant. [10]  Model [240AC-NA]',disabled=False, style=style1, layout=layout1)
cant11=widgets.Checkbox(value=True,description='Cant. [11]  Model [240AC-NG]',disabled=False, style=style1, layout=layout1)
cant12=widgets.Checkbox(value=True,description='Cant. [12]  Model [240AC-NN]',disabled=False, style=style1, layout=layout1)
cant13=widgets.Checkbox(value=True,description='Cant. [13]  Model [240AC-SG]',disabled=False, style=style1, layout=layout1)
cant14=widgets.Checkbox(value=True,description='Cant. [14]  Model [3XC-GG-A]',disabled=False, style=style1, layout=layout1)
cant15=widgets.Checkbox(value=True,description='Cant. [15]  Model [3XC-GG-B]',disabled=False, style=style1, layout=layout1)
cant16=widgets.Checkbox(value=True,description='Cant. [16]  Model [3XC-GG-C]',disabled=False, style=style1, layout=layout1)
cant17=widgets.Checkbox(value=True,description='Cant. [17]  Model [3XC-NA-B]',disabled=False, style=style1, layout=layout1)
cant18=widgets.Checkbox(value=True,description='Cant. [18]  Model [3XC-NA-C]',disabled=False, style=style1, layout=layout1)
cant19=widgets.Checkbox(value=True,description='Cant. [19]  Model [3XC-NN-A]',disabled=False, style=style1, layout=layout1)
cant20=widgets.Checkbox(value=True,description='Cant. [20]  Model [3XC-NN-B]',disabled=False, style=style1, layout=layout1)
cant21=widgets.Checkbox(value=True,description='Cant. [21]  Model [3XC-NN-C]',disabled=False, style=style1, layout=layout1)
cant22=widgets.Checkbox(value=True,description='Cant. [22]  Model [4XC-GG-B]',disabled=False, style=style1, layout=layout1)
cant23=widgets.Checkbox(value=True,description='Cant. [23]  Model [4XC-GG-C]',disabled=False, style=style1, layout=layout1)
cant24=widgets.Checkbox(value=True,description='Cant. [24]  Model [4XC-NN-B]',disabled=False, style=style1, layout=layout1)
cant25=widgets.Checkbox(value=True,description='Cant. [25]  Model [4XC-NN-C]',disabled=False, style=style1, layout=layout1)
cant26=widgets.Checkbox(value=True,description='Cant. [26]  Model [55AC-NA]',disabled=False, style=style1, layout=layout1)
cant27=widgets.Checkbox(value=True,description='Cant. [27]  Model [55AC-NN]',disabled=False, style=style1, layout=layout1)
cant28=widgets.Checkbox(value=True,description='Cant. [28]  Model [All-In-One-A]',disabled=False, style=style1, layout=layout1)
cant29=widgets.Checkbox(value=True,description='Cant. [29]  Model [All-In-One-Al-A]',disabled=False, style=style1, layout=layout1)
cant30=widgets.Checkbox(value=True,description='Cant. [30]  Model [All-In-One-Al-B]',disabled=False, style=style1, layout=layout1)
cant31=widgets.Checkbox(value=True,description='Cant. [31]  Model [All-In-One-Al-C]',disabled=False, style=style1, layout=layout1)
cant32=widgets.Checkbox(value=True,description='Cant. [32]  Model [All-In-One-Al-D]',disabled=False, style=style1, layout=layout1)
cant33=widgets.Checkbox(value=True,description='Cant. [33]  Model [All-In-One-B]',disabled=False, style=style1, layout=layout1)
cant34=widgets.Checkbox(value=True,description='Cant. [34]  Model [All-In-One-C]',disabled=False, style=style1, layout=layout1)
cant35=widgets.Checkbox(value=True,description='Cant. [35]  Model [All-In-One-D]',disabled=False, style=style1, layout=layout1)
cant36=widgets.Checkbox(value=True,description='Cant. [36]  Model [All-In-One-DD-A]',disabled=False, style=style1, layout=layout1)
cant37=widgets.Checkbox(value=True,description='Cant. [37]  Model [All-In-One-DD-B]',disabled=False, style=style1, layout=layout1)
cant38=widgets.Checkbox(value=True,description='Cant. [38]  Model [All-In-One-DD-C]',disabled=False, style=style1, layout=layout1)
cant39=widgets.Checkbox(value=True,description='Cant. [39]  Model [All-In-One-DD-D]',disabled=False, style=style1, layout=layout1)
cant40=widgets.Checkbox(value=True,description='Cant. [40]  Model [AR10-NCHR]',disabled=False, style=style1, layout=layout1)
cant41=widgets.Checkbox(value=True,description='Cant. [41]  Model [AR5-NCHR]',disabled=False, style=style1, layout=layout1)
cant42=widgets.Checkbox(value=True,description='Cant. [42]  Model [AR5T-NCHR]',disabled=False, style=style1, layout=layout1)
cant43=widgets.Checkbox(value=True,description='Cant. [43]  Model [ARROW-FM]',disabled=False, style=style1, layout=layout1)
cant44=widgets.Checkbox(value=True,description='Cant. [44]  Model [ARROW-FMR]',disabled=False, style=style1, layout=layout1)
cant45=widgets.Checkbox(value=True,description='Cant. [45]  Model [ARROW-NC]',disabled=False, style=style1, layout=layout1)
cant46=widgets.Checkbox(value=True,description='Cant. [46]  Model [ARROW-NCR]',disabled=False, style=style1, layout=layout1)
cant47=widgets.Checkbox(value=True,description='Cant. [47]  Model [ARROW-UHFAuD]',disabled=False, style=style1, layout=layout1)
cant48=widgets.Checkbox(value=True,description='Cant. [48]  Model [ATEC-CONT]',disabled=False, style=style1, layout=layout1)
cant49=widgets.Checkbox(value=True,description='Cant. [49]  Model [ATEC-FM]',disabled=False, style=style1, layout=layout1)
cant50=widgets.Checkbox(value=True,description='Cant. [50]  Model [ATEC-FMAu]',disabled=False, style=style1, layout=layout1)
cant51=widgets.Checkbox(value=True,description='Cant. [51]  Model [ATEC-NC]',disabled=False, style=style1, layout=layout1)
cant52=widgets.Checkbox(value=True,description='Cant. [52]  Model [biotool cell]',disabled=False, style=style1, layout=layout1)
cant53=widgets.Checkbox(value=True,description='Cant. [53]  Model [biotool cell XXL]',disabled=False, style=style1, layout=layout1)
cant54=widgets.Checkbox(value=True,description='Cant. [54]  Model [CONTR]',disabled=False, style=style1, layout=layout1)
cant55=widgets.Checkbox(value=True,description='Cant. [55]  Model [DT-NCHR]',disabled=False, style=style1, layout=layout1)
cant56=widgets.Checkbox(value=True,description='Cant. [56]  Model [EBD-NCH]',disabled=False, style=style1, layout=layout1)
cant57=widgets.Checkbox(value=True,description='Cant. [57]  Model [FASTSCAN-D-SS]',disabled=False, style=style1, layout=layout1)
cant58=widgets.Checkbox(value=True,description='Cant. [58]  Model [FESP]',disabled=False, style=style1, layout=layout1)
cant59=widgets.Checkbox(value=True,description='Cant. [59]  Model [FM]',disabled=False, style=style1, layout=layout1)
cant60=widgets.Checkbox(value=True,description='Cant. [60]  Model [FMR]',disabled=False, style=style1, layout=layout1)
cant61=widgets.Checkbox(value=True,description='Cant. [61]  Model [HiRes-C18/Cr-Au]',disabled=False, style=style1, layout=layout1)
cant62=widgets.Checkbox(value=True,description='Cant. [62]  Model [HQ:CSC17]',disabled=False, style=style1, layout=layout1)
cant63=widgets.Checkbox(value=True,description='Cant. [63]  Model [HQ:CSC37-A]',disabled=False, style=style1, layout=layout1)
cant64=widgets.Checkbox(value=True,description='Cant. [64]  Model [HQ:CSC37-B]',disabled=False, style=style1, layout=layout1)
cant65=widgets.Checkbox(value=True,description='Cant. [65]  Model [HQ:CSC37-C]',disabled=False, style=style1, layout=layout1)
cant66=widgets.Checkbox(value=True,description='Cant. [66]  Model [HQ:CSC38-A]',disabled=False, style=style1, layout=layout1)
cant67=widgets.Checkbox(value=True,description='Cant. [67]  Model [HQ:CSC38-B]',disabled=False, style=style1, layout=layout1)
cant68=widgets.Checkbox(value=True,description='Cant. [68]  Model [HQ:CSC38-C]',disabled=False, style=style1, layout=layout1)
cant69=widgets.Checkbox(value=True,description='Cant. [69]  Model [HQ:NSC14/Al BS]',disabled=False, style=style1, layout=layout1)
cant70=widgets.Checkbox(value=True,description='Cant. [70]  Model [HQ:NSC14/Cr-Au BS]',disabled=False, style=style1, layout=layout1)
cant71=widgets.Checkbox(value=True,description='Cant. [71]  Model [HQ:NSC14/No Al]',disabled=False, style=style1, layout=layout1)
cant72=widgets.Checkbox(value=True,description='Cant. [72]  Model [HQ:NSC15/Al BS]',disabled=False, style=style1, layout=layout1)
cant73=widgets.Checkbox(value=True,description='Cant. [73]  Model [HQ:NSC15/Cr-Au BS]',disabled=False, style=style1, layout=layout1)
cant74=widgets.Checkbox(value=True,description='Cant. [74]  Model [HQ:NSC15/No Al]',disabled=False, style=style1, layout=layout1)
cant75=widgets.Checkbox(value=True,description='Cant. [75]  Model [HQ:NSC16/Al BS]',disabled=False, style=style1, layout=layout1)
cant76=widgets.Checkbox(value=True,description='Cant. [76]  Model [HQ:NSC16/Cr-Au BS]',disabled=False, style=style1, layout=layout1)
cant77=widgets.Checkbox(value=True,description='Cant. [77]  Model [HQ:NSC16/No Al]',disabled=False, style=style1, layout=layout1)
cant78=widgets.Checkbox(value=True,description='Cant. [78]  Model [HQ:NSC18/Al BS]',disabled=False, style=style1, layout=layout1)
cant79=widgets.Checkbox(value=True,description='Cant. [79]  Model [HQ:NSC18/Cr-Au BS]',disabled=False, style=style1, layout=layout1)
cant80=widgets.Checkbox(value=True,description='Cant. [80]  Model [HQ:NSC18/No Al]',disabled=False, style=style1, layout=layout1)
cant81=widgets.Checkbox(value=True,description='Cant. [81]  Model [HQ:NSC19/Al BS]',disabled=False, style=style1, layout=layout1)
cant82=widgets.Checkbox(value=True,description='Cant. [82]  Model [HQ:NSC19/Cr-Au]',disabled=False, style=style1, layout=layout1)
cant83=widgets.Checkbox(value=True,description='Cant. [83]  Model [HQ:NSC19/No Al]',disabled=False, style=style1, layout=layout1)
cant84=widgets.Checkbox(value=True,description='Cant. [84]  Model [HQ:NSC35/Al BS-A]',disabled=False, style=style1, layout=layout1)
cant85=widgets.Checkbox(value=True,description='Cant. [85]  Model [HQ:NSC35/Al BS-B]',disabled=False, style=style1, layout=layout1)
cant86=widgets.Checkbox(value=True,description='Cant. [86]  Model [HQ:NSC35/Al BS-C]',disabled=False, style=style1, layout=layout1)
cant87=widgets.Checkbox(value=True,description='Cant. [87]  Model [HQ:NSC35/Cr-Au BS-A]',disabled=False, style=style1, layout=layout1)
cant88=widgets.Checkbox(value=True,description='Cant. [88]  Model [HQ:NSC35/Cr-Au BS-B]',disabled=False, style=style1, layout=layout1)
cant89=widgets.Checkbox(value=True,description='Cant. [89]  Model [HQ:NSC35/Cr-Au BS-C]',disabled=False, style=style1, layout=layout1)
cant90=widgets.Checkbox(value=True,description='Cant. [90]  Model [HQ:NSC35/Cr-Au-A]',disabled=False, style=style1, layout=layout1)
cant91=widgets.Checkbox(value=True,description='Cant. [91]  Model [HQ:NSC35/Cr-Au-B]',disabled=False, style=style1, layout=layout1)
cant92=widgets.Checkbox(value=True,description='Cant. [92]  Model [HQ:NSC35/Cr-Au-C]',disabled=False, style=style1, layout=layout1)
cant93=widgets.Checkbox(value=True,description='Cant. [93]  Model [HQ:NSC35/No Al-A]',disabled=False, style=style1, layout=layout1)
cant94=widgets.Checkbox(value=True,description='Cant. [94]  Model [HQ:NSC35/No Al-B]',disabled=False, style=style1, layout=layout1)
cant95=widgets.Checkbox(value=True,description='Cant. [95]  Model [HQ:NSC35/No Al-C]',disabled=False, style=style1, layout=layout1)
cant96=widgets.Checkbox(value=True,description='Cant. [96]  Model [HQ:NSC36/Al BS-A]',disabled=False, style=style1, layout=layout1)
cant97=widgets.Checkbox(value=True,description='Cant. [97]  Model [HQ:NSC36/Al BS-B]',disabled=False, style=style1, layout=layout1)
cant98=widgets.Checkbox(value=True,description='Cant. [98]  Model [HQ:NSC36/Al BS-C]',disabled=False, style=style1, layout=layout1)
cant99=widgets.Checkbox(value=True,description='Cant. [99]  Model [HQ:NSC36/Cr-Au BS-A]',disabled=False, style=style1, layout=layout1)
cant100=widgets.Checkbox(value=True,description='Cant. [100]  Model [HQ:NSC36/Cr-Au BS-B]',disabled=False, style=style1, layout=layout1)
cant101=widgets.Checkbox(value=True,description='Cant. [101]  Model [HQ:NSC36/Cr-Au BS-C]',disabled=False, style=style1, layout=layout1)
cant102=widgets.Checkbox(value=True,description='Cant. [102]  Model [HQ:NSC36/Cr-Au-A]',disabled=False, style=style1, layout=layout1)
cant103=widgets.Checkbox(value=True,description='Cant. [103]  Model [HQ:NSC36/Cr-Au-B]',disabled=False, style=style1, layout=layout1)
cant104=widgets.Checkbox(value=True,description='Cant. [104]  Model [HQ:NSC36/Cr-Au-C]',disabled=False, style=style1, layout=layout1)
cant105=widgets.Checkbox(value=True,description='Cant. [105]  Model [HQ:NSC36/No Al-A]',disabled=False, style=style1, layout=layout1)
cant106=widgets.Checkbox(value=True,description='Cant. [106]  Model [HQ:NSC36/No Al-B]',disabled=False, style=style1, layout=layout1)
cant107=widgets.Checkbox(value=True,description='Cant. [107]  Model [HQ:NSC36/No Al-C]',disabled=False, style=style1, layout=layout1)
cant108=widgets.Checkbox(value=True,description='Cant. [108]  Model [HQ:XSC11/Al BS-A]',disabled=False, style=style1, layout=layout1)
cant109=widgets.Checkbox(value=True,description='Cant. [109]  Model [HQ:XSC11/Al BS-B]',disabled=False, style=style1, layout=layout1)
cant110=widgets.Checkbox(value=True,description='Cant. [110]  Model [HQ:XSC11/Al BS-C]',disabled=False, style=style1, layout=layout1)
cant111=widgets.Checkbox(value=True,description='Cant. [111]  Model [HQ:XSC11/Al BS-D]',disabled=False, style=style1, layout=layout1)
cant112=widgets.Checkbox(value=True,description='Cant. [112]  Model [HQ:XSC11/No Al-A]',disabled=False, style=style1, layout=layout1)
cant113=widgets.Checkbox(value=True,description='Cant. [113]  Model [HQ:XSC11/No Al-B]',disabled=False, style=style1, layout=layout1)
cant114=widgets.Checkbox(value=True,description='Cant. [114]  Model [HQ:XSC11/No Al-C]',disabled=False, style=style1, layout=layout1)
cant115=widgets.Checkbox(value=True,description='Cant. [115]  Model [HQ:XSC11/No Al-D]',disabled=False, style=style1, layout=layout1)
cant116=widgets.Checkbox(value=True,description='Cant. [116]  Model [LTESP]',disabled=False, style=style1, layout=layout1)
cant117=widgets.Checkbox(value=True,description='Cant. [117]  Model [MSNL-10]',disabled=False, style=style1, layout=layout1)
cant118=widgets.Checkbox(value=True,description='Cant. [118]  Model [Multi75-G]',disabled=False, style=style1, layout=layout1)
cant119=widgets.Checkbox(value=True,description='Cant. [119]  Model [Multi75Al-G]',disabled=False, style=style1, layout=layout1)
cant120=widgets.Checkbox(value=True,description='Cant. [120]  Model [Multi75GB-G]',disabled=False, style=style1, layout=layout1)
cant121=widgets.Checkbox(value=True,description='Cant. [121]  Model [Multi75GD-G]',disabled=False, style=style1, layout=layout1)
cant122=widgets.Checkbox(value=True,description='Cant. [122]  Model [NCH]',disabled=False, style=style1, layout=layout1)
cant123=widgets.Checkbox(value=True,description='Cant. [123]  Model [NCHR]',disabled=False, style=style1, layout=layout1)
cant124=widgets.Checkbox(value=True,description='Cant. [124]  Model [NCL]',disabled=False, style=style1, layout=layout1)
cant125=widgets.Checkbox(value=True,description='Cant. [125]  Model [NCLR]',disabled=False, style=style1, layout=layout1)
cant126=widgets.Checkbox(value=True,description='Cant. [126]  Model [NCST]',disabled=False, style=style1, layout=layout1)
cant127=widgets.Checkbox(value=True,description='Cant. [127]  Model [NCSTR]',disabled=False, style=style1, layout=layout1)
cant128=widgets.Checkbox(value=True,description='Cant. [128]  Model [NSC36 / Al BS-A]',disabled=False, style=style1, layout=layout1)
cant129=widgets.Checkbox(value=True,description='Cant. [129]  Model [NSC36 / Al BS-B]',disabled=False, style=style1, layout=layout1)
cant130=widgets.Checkbox(value=True,description='Cant. [130]  Model [NSC36 / Al BS-C]',disabled=False, style=style1, layout=layout1)
cant131=widgets.Checkbox(value=True,description='Cant. [131]  Model [NW-CDT-NCHR]',disabled=False, style=style1, layout=layout1)
cant132=widgets.Checkbox(value=True,description='Cant. [132]  Model [NW-CDT-NCLR]',disabled=False, style=style1, layout=layout1)
cant133=widgets.Checkbox(value=True,description='Cant. [133]  Model [NW-SSS-NCH]',disabled=False, style=style1, layout=layout1)
cant134=widgets.Checkbox(value=True,description='Cant. [134]  Model [NW-SSS-NCL]',disabled=False, style=style1, layout=layout1)
cant135=widgets.Checkbox(value=True,description='Cant. [135]  Model [NW-SSS-SEIH]',disabled=False, style=style1, layout=layout1)
cant136=widgets.Checkbox(value=True,description='Cant. [136]  Model [OLTESPA]',disabled=False, style=style1, layout=layout1)
cant137=widgets.Checkbox(value=True,description='Cant. [137]  Model [OTESPA]',disabled=False, style=style1, layout=layout1)
cant138=widgets.Checkbox(value=True,description='Cant. [138]  Model [PEAKFORCE-HIRS-F-A]',disabled=False, style=style1, layout=layout1)
cant139=widgets.Checkbox(value=True,description='Cant. [139]  Model [PEAKFORCE-HIRS-F-B]',disabled=False, style=style1, layout=layout1)
cant140=widgets.Checkbox(value=True,description='Cant. [140]  Model [PEAKFORCE-HIRS-SSB]',disabled=False, style=style1, layout=layout1)
cant141=widgets.Checkbox(value=True,description='Cant. [141]  Model [PNP-DB-A]',disabled=False, style=style1, layout=layout1)
cant142=widgets.Checkbox(value=True,description='Cant. [142]  Model [PNP-DB-B]',disabled=False, style=style1, layout=layout1)
cant143=widgets.Checkbox(value=True,description='Cant. [143]  Model [PNP-TR-A]',disabled=False, style=style1, layout=layout1)
cant144=widgets.Checkbox(value=True,description='Cant. [144]  Model [PNP-TR-Au-A]',disabled=False, style=style1, layout=layout1)
cant145=widgets.Checkbox(value=True,description='Cant. [145]  Model [PNP-TR-Au-B]',disabled=False, style=style1, layout=layout1)
cant146=widgets.Checkbox(value=True,description='Cant. [146]  Model [PNP-TR-B]',disabled=False, style=style1, layout=layout1)
cant147=widgets.Checkbox(value=True,description='Cant. [147]  Model [PNP-TRS]',disabled=False, style=style1, layout=layout1)
cant148=widgets.Checkbox(value=True,description='Cant. [148]  Model [PPP-CONT]',disabled=False, style=style1, layout=layout1)
cant149=widgets.Checkbox(value=True,description='Cant. [149]  Model [PPP-CONT-SC]',disabled=False, style=style1, layout=layout1)
cant150=widgets.Checkbox(value=True,description='Cant. [150]  Model [PPP-CONTAuD]',disabled=False, style=style1, layout=layout1)
cant151=widgets.Checkbox(value=True,description='Cant. [151]  Model [PPP-CONTR]',disabled=False, style=style1, layout=layout1)
cant152=widgets.Checkbox(value=True,description='Cant. [152]  Model [PPP-CONTSC]',disabled=False, style=style1, layout=layout1)
cant153=widgets.Checkbox(value=True,description='Cant. [153]  Model [PPP-CONTSCAuD]',disabled=False, style=style1, layout=layout1)
cant154=widgets.Checkbox(value=True,description='Cant. [154]  Model [PPP-CONTSCPt]',disabled=False, style=style1, layout=layout1)
cant155=widgets.Checkbox(value=True,description='Cant. [155]  Model [PPP-CONTSCR]',disabled=False, style=style1, layout=layout1)
cant156=widgets.Checkbox(value=True,description='Cant. [156]  Model [PPP-FM]',disabled=False, style=style1, layout=layout1)
cant157=widgets.Checkbox(value=True,description='Cant. [157]  Model [PPP-FMAu]',disabled=False, style=style1, layout=layout1)
cant158=widgets.Checkbox(value=True,description='Cant. [158]  Model [PPP-FMAuD]',disabled=False, style=style1, layout=layout1)
cant159=widgets.Checkbox(value=True,description='Cant. [159]  Model [PPP-FMR]',disabled=False, style=style1, layout=layout1)
cant160=widgets.Checkbox(value=True,description='Cant. [160]  Model [PPP-LFM-R]',disabled=False, style=style1, layout=layout1)
cant161=widgets.Checkbox(value=True,description='Cant. [161]  Model [PPP-LFMR]',disabled=False, style=style1, layout=layout1)
cant162=widgets.Checkbox(value=True,description='Cant. [162]  Model [PPP-NCH]',disabled=False, style=style1, layout=layout1)
cant163=widgets.Checkbox(value=True,description='Cant. [163]  Model [PPP-NCHAuD]',disabled=False, style=style1, layout=layout1)
cant164=widgets.Checkbox(value=True,description='Cant. [164]  Model [PPP-NCHR]',disabled=False, style=style1, layout=layout1)
cant165=widgets.Checkbox(value=True,description='Cant. [165]  Model [PPP-NCL]',disabled=False, style=style1, layout=layout1)
cant166=widgets.Checkbox(value=True,description='Cant. [166]  Model [PPP-NCLAuD]',disabled=False, style=style1, layout=layout1)
cant167=widgets.Checkbox(value=True,description='Cant. [167]  Model [PPP-NCLR]',disabled=False, style=style1, layout=layout1)
cant168=widgets.Checkbox(value=True,description='Cant. [168]  Model [PPP-NCST]',disabled=False, style=style1, layout=layout1)
cant169=widgets.Checkbox(value=True,description='Cant. [169]  Model [PPP-NCSTAuD]',disabled=False, style=style1, layout=layout1)
cant170=widgets.Checkbox(value=True,description='Cant. [170]  Model [PPP-NCSTR]',disabled=False, style=style1, layout=layout1)
cant171=widgets.Checkbox(value=True,description='Cant. [171]  Model [PPP-QFMR]',disabled=False, style=style1, layout=layout1)
cant172=widgets.Checkbox(value=True,description='Cant. [172]  Model [PPP-QFMRAu]',disabled=False, style=style1, layout=layout1)
cant173=widgets.Checkbox(value=True,description='Cant. [173]  Model [PPP-QNCHR]',disabled=False, style=style1, layout=layout1)
cant174=widgets.Checkbox(value=True,description='Cant. [174]  Model [PPP-RT-CONTR]',disabled=False, style=style1, layout=layout1)
cant175=widgets.Checkbox(value=True,description='Cant. [175]  Model [PPP-RT-FMR]',disabled=False, style=style1, layout=layout1)
cant176=widgets.Checkbox(value=True,description='Cant. [176]  Model [PPP-RT-FMRAu]',disabled=False, style=style1, layout=layout1)
cant177=widgets.Checkbox(value=True,description='Cant. [177]  Model [PPP-RT-NCHR]',disabled=False, style=style1, layout=layout1)
cant178=widgets.Checkbox(value=True,description='Cant. [178]  Model [PPP-SEIH]',disabled=False, style=style1, layout=layout1)
cant179=widgets.Checkbox(value=True,description='Cant. [179]  Model [PPP-SEIHR]',disabled=False, style=style1, layout=layout1)
cant180=widgets.Checkbox(value=True,description='Cant. [180]  Model [PPP-XY-CONTR]',disabled=False, style=style1, layout=layout1)
cant181=widgets.Checkbox(value=True,description='Cant. [181]  Model [PPP-XYNCHR]',disabled=False, style=style1, layout=layout1)
cant182=widgets.Checkbox(value=True,description='Cant. [182]  Model [PPP-XYNCSTR]',disabled=False, style=style1, layout=layout1)
cant183=widgets.Checkbox(value=True,description='Cant. [183]  Model [PPP-ZEI-LR]',disabled=False, style=style1, layout=layout1)
cant184=widgets.Checkbox(value=True,description='Cant. [184]  Model [PPP-ZEIHR]',disabled=False, style=style1, layout=layout1)
cant185=widgets.Checkbox(value=True,description='Cant. [185]  Model [PR-SSP10]',disabled=False, style=style1, layout=layout1)
cant186=widgets.Checkbox(value=True,description='Cant. [186]  Model [Q-WM190-SSS]',disabled=False, style=style1, layout=layout1)
cant187=widgets.Checkbox(value=True,description='Cant. [187]  Model [qp-BioAC-A]',disabled=False, style=style1, layout=layout1)
cant188=widgets.Checkbox(value=True,description='Cant. [188]  Model [qp-BioAC-B]',disabled=False, style=style1, layout=layout1)
cant189=widgets.Checkbox(value=True,description='Cant. [189]  Model [qp-BioAC-C]',disabled=False, style=style1, layout=layout1)
cant190=widgets.Checkbox(value=True,description='Cant. [190]  Model [qp-BioAC-CI-A]',disabled=False, style=style1, layout=layout1)
cant191=widgets.Checkbox(value=True,description='Cant. [191]  Model [qp-BioAC-CI-B]',disabled=False, style=style1, layout=layout1)
cant192=widgets.Checkbox(value=True,description='Cant. [192]  Model [qp-BioAC-CI-C]',disabled=False, style=style1, layout=layout1)
cant193=widgets.Checkbox(value=True,description='Cant. [193]  Model [qp-BioT-A]',disabled=False, style=style1, layout=layout1)
cant194=widgets.Checkbox(value=True,description='Cant. [194]  Model [qp-BioT-B]',disabled=False, style=style1, layout=layout1)
cant195=widgets.Checkbox(value=True,description='Cant. [195]  Model [qp-CONT]',disabled=False, style=style1, layout=layout1)
cant196=widgets.Checkbox(value=True,description='Cant. [196]  Model [qp-fast-A]',disabled=False, style=style1, layout=layout1)
cant197=widgets.Checkbox(value=True,description='Cant. [197]  Model [qp-fast-B]',disabled=False, style=style1, layout=layout1)
cant198=widgets.Checkbox(value=True,description='Cant. [198]  Model [qp-fast-C]',disabled=False, style=style1, layout=layout1)
cant199=widgets.Checkbox(value=True,description='Cant. [199]  Model [qp-HBC]',disabled=False, style=style1, layout=layout1)
cant200=widgets.Checkbox(value=True,description='Cant. [200]  Model [qp-SCONT]',disabled=False, style=style1, layout=layout1)
cant201=widgets.Checkbox(value=True,description='Cant. [201]  Model [SAA-HPI-SS]',disabled=False, style=style1, layout=layout1)
cant202=widgets.Checkbox(value=True,description='Cant. [202]  Model [SCANASYST-AIR]',disabled=False, style=style1, layout=layout1)
cant203=widgets.Checkbox(value=True,description='Cant. [203]  Model [SCANASYST-FLUID+]',disabled=False, style=style1, layout=layout1)
cant204=widgets.Checkbox(value=True,description='Cant. [204]  Model [SD-CONT-SiN*]',disabled=False, style=style1, layout=layout1)
cant205=widgets.Checkbox(value=True,description='Cant. [205]  Model [SD-P-CONT]',disabled=False, style=style1, layout=layout1)
cant206=widgets.Checkbox(value=True,description='Cant. [206]  Model [SD-PXL-CONT]',disabled=False, style=style1, layout=layout1)
cant207=widgets.Checkbox(value=True,description='Cant. [207]  Model [SD-PXL-CONT-SC]',disabled=False, style=style1, layout=layout1)
cant208=widgets.Checkbox(value=True,description='Cant. [208]  Model [SD-T1L450B]',disabled=False, style=style1, layout=layout1)
cant209=widgets.Checkbox(value=True,description='Cant. [209]  Model [SD-T5L450B]',disabled=False, style=style1, layout=layout1)
cant210=widgets.Checkbox(value=True,description='Cant. [210]  Model [SEIHR]',disabled=False, style=style1, layout=layout1)
cant211=widgets.Checkbox(value=True,description='Cant. [211]  Model [SiNi-A]',disabled=False, style=style1, layout=layout1)
cant212=widgets.Checkbox(value=True,description='Cant. [212]  Model [SiNi-B]',disabled=False, style=style1, layout=layout1)
cant213=widgets.Checkbox(value=True,description='Cant. [213]  Model [SNL-10]',disabled=False, style=style1, layout=layout1)
cant214=widgets.Checkbox(value=True,description='Cant. [214]  Model [SNL-W]',disabled=False, style=style1, layout=layout1)
cant215=widgets.Checkbox(value=True,description='Cant. [215]  Model [SSS-FM]',disabled=False, style=style1, layout=layout1)
cant216=widgets.Checkbox(value=True,description='Cant. [216]  Model [SSS-FMR]',disabled=False, style=style1, layout=layout1)
cant217=widgets.Checkbox(value=True,description='Cant. [217]  Model [SSS-NCH]',disabled=False, style=style1, layout=layout1)
cant218=widgets.Checkbox(value=True,description='Cant. [218]  Model [SSS-NCHR]',disabled=False, style=style1, layout=layout1)
cant219=widgets.Checkbox(value=True,description='Cant. [219]  Model [SSS-NCL]',disabled=False, style=style1, layout=layout1)
cant220=widgets.Checkbox(value=True,description='Cant. [220]  Model [SSS-NCLR]',disabled=False, style=style1, layout=layout1)
cant221=widgets.Checkbox(value=True,description='Cant. [221]  Model [SSS-SEIH]',disabled=False, style=style1, layout=layout1)
cant222=widgets.Checkbox(value=True,description='Cant. [222]  Model [SSS-SEIHR]',disabled=False, style=style1, layout=layout1)
cant223=widgets.Checkbox(value=True,description='Cant. [223]  Model [Tap150-G]',disabled=False, style=style1, layout=layout1)
cant224=widgets.Checkbox(value=True,description='Cant. [224]  Model [Tap150Al-G]',disabled=False, style=style1, layout=layout1)
cant225=widgets.Checkbox(value=True,description='Cant. [225]  Model [Tap150GD-G]',disabled=False, style=style1, layout=layout1)
cant226=widgets.Checkbox(value=True,description='Cant. [226]  Model [Tap190-G]',disabled=False, style=style1, layout=layout1)
cant227=widgets.Checkbox(value=True,description='Cant. [227]  Model [Tap190Al-G]',disabled=False, style=style1, layout=layout1)
cant228=widgets.Checkbox(value=True,description='Cant. [228]  Model [Tap190GD-G]',disabled=False, style=style1, layout=layout1)
cant229=widgets.Checkbox(value=True,description='Cant. [229]  Model [Tap300-G]',disabled=False, style=style1, layout=layout1)
cant230=widgets.Checkbox(value=True,description='Cant. [230]  Model [Tap300Al-G]',disabled=False, style=style1, layout=layout1)
cant231=widgets.Checkbox(value=True,description='Cant. [231]  Model [Tap300GD-G]',disabled=False, style=style1, layout=layout1)
cant232=widgets.Checkbox(value=True,description='Cant. [232]  Model [TESP]',disabled=False, style=style1, layout=layout1)
cant233=widgets.Checkbox(value=True,description='Cant. [233]  Model [TESP-SS]',disabled=False, style=style1, layout=layout1)
cant234=widgets.Checkbox(value=True,description='Cant. [234]  Model [TESPA]',disabled=False, style=style1, layout=layout1)
cant235=widgets.Checkbox(value=True,description='Cant. [235]  Model [USC-F0.3-k0.3]',disabled=False, style=style1, layout=layout1)
cant236=widgets.Checkbox(value=True,description='Cant. [236]  Model [USC-F1.2-k0.15]',disabled=False, style=style1, layout=layout1)
cant237=widgets.Checkbox(value=True,description='Cant. [237]  Model [USC-F1.5-k0.6]',disabled=False, style=style1, layout=layout1)
cant238=widgets.Checkbox(value=True,description='Cant. [238]  Model [XCS11-A]',disabled=False, style=style1, layout=layout1)
cant239=widgets.Checkbox(value=True,description='Cant. [239]  Model [XCS11-B]',disabled=False, style=style1, layout=layout1)
cant240=widgets.Checkbox(value=True,description='Cant. [240]  Model [XNC12/Cr-Au BS-A]',disabled=False, style=style1, layout=layout1)
cant241=widgets.Checkbox(value=True,description='Cant. [241]  Model [XNC12/Cr-Au BS-B]',disabled=False, style=style1, layout=layout1)
cant242=widgets.Checkbox(value=True,description='Cant. [242]  Model [ZEIHR]',disabled=False, style=style1, layout=layout1)




#=========POSITION FOR BOTOMS========
style = {'description_width': '300px'}
layout = {'width': '600px'}
    
#===================VARIABLES====================
a0=widgets.FloatText(value=10, step=0.1, description='Amplitude [nm]  ',disabled=False, style=style, layout=layout)
Es=widgets.FloatText(value=200,step=0.1, description='Young modulus sample [MPa]  ',disabled=False, style=style, layout=layout)
q=widgets.FloatText(value=25,step=0.1, description='Quality factor [Adim]  ',disabled=False, style=style, layout=layout)
RH=widgets.FloatText(value=20,step=0.1, description='Relative Humidity [%] ',disabled=False, style=style, layout=layout)
#display(a0,Es,q,RH)
#===================FORCES====================

h=widgets.Checkbox(value=True,description='Hertz contact mechanics',disabled=True, style=style, layout=layout)
w=widgets.Checkbox(value=False,description='Van der Waals',disabled=False, style=style, layout=layout)
d=widgets.Checkbox(value=False,description='Derjaguin-Landau-Verbey-Overbeek (DLVO)',disabled=False, style=style, layout=layout)
v=widgets.Checkbox(value=False,description='Linear viscosity',disabled=False, style=style, layout=layout)
c=widgets.Checkbox(value=False,description='Capillary',disabled=False, style=style, layout=layout)
l=widgets.Checkbox(value=False,description='Lennard-Jones',disabled=True, style=style, layout=layout)
#display(h,w,d,v,c,l)
#===================NUMERICAL INTEGRATION=================
nper=widgets.FloatText(value=100, step=1, description='Number of periods  ',disabled=False, style=style, layout=layout)
npp=widgets.FloatText(value=128, step=1, description='Number of points per period  ',disabled=False, style=style, layout=layout)
nperfin=widgets.FloatText(value=8, step=1, description='Interval of periods  ',disabled=False, style=style, layout=layout)
zcmin=widgets.FloatText(value=0, step=0.1, description='Minimum distance [nm]  ',disabled=False, style=style, layout=layout)
dzc=widgets.FloatText(value=0.1, step=0.01, description='Average tip–surface distance step [nm]  ',disabled=False, style=style, layout=layout)
fixzc=widgets.FloatText(value=3.5, step=0.1, description='Distance fixed [nm]  ',disabled=False, style=style, layout=layout)
#display(nper,npp,nperfin,zcmin,dzc,fixzc)
#===================PARAMETERS=================
ham=widgets.FloatText(value=2, step=0.01, description='Hamaker Const [*10^-20 J]  ',disabled=False, style=style, layout=layout)
epsilon1=widgets.FloatText(value=0, step=0.01, description='LR Disp. [Adim]  ',disabled=False, style=style, layout=layout)
eps=widgets.FloatText(value=79, step=0.01, description='Relative permeability [C^2/N m^2]  ',disabled=False, style=style, layout=layout)
eps0=widgets.FloatText(value=8.85, step=0.01, description='Vacuum permeability [C^2/N m^2]  ',disabled=False, style=style, layout=layout)
sigmat=widgets.FloatText(value=25, step=0.01, description='Tip-Surface loading density [mC/m^2]  ',disabled=False, style=style, layout=layout)
sigmas=widgets.FloatText(value=16, step=0.01, description='Sample surface charge density [mC/m^2]  ',disabled=False, style=style, layout=layout)
debye=widgets.FloatText(value=1, step=0.01, description='Debye lengths [nm]  ' ,disabled=False, style=style, layout=layout)
eta=widgets.FloatText(value=100, step=0.01, description='Viscosity [Pa s]  ',disabled=False, style=style, layout=layout)
epsilon2=widgets.FloatText(value=10, step=0.01, description='Surface energy of solid-liquid  [mN/m]  ',disabled=False, style=style, layout=layout)
ljmin=widgets.FloatText(value=0, step=0.01, description='Epsilon force LJ [nN]  ',disabled=False, style=style, layout=layout)
lenght=widgets.FloatText(value=0, step=0.01, description='Sigma force LJ [nm]  ',disabled=False, style=style, layout=layout)
#===================EXECUTE=================
hilos = widgets.FloatText(value=10, step=1, description='Number of Threads  ',disabled=False, style=style, layout=layout)
run = widgets.Button(description='EXECUTE',disabled=False,button_style='danger',tooltip='Click me', icon='check',style=style, layout=layout)

#===================DATA BASE ===================
database = pd.read_csv("base_datos.csv")
dbb = widgets.Button(description='Data Base',disabled=False,style=style, layout=layout)
def databasef (x=False):
    if x:
        display(database)
#==================PYDAMPF LOGO===========
file = open("logo.png", "rb")
image = file.read()
im = widgets.Image(value=image,format='png',width=300,height=300)

#=================TABS==============
right_box = widgets.VBox([im])

left_box0_0 = widgets.VBox([cant1,cant2,cant3,cant4,cant5,cant6,cant7,cant8,cant9,cant10,cant11,cant12,cant13,cant14,cant15,cant16,cant17,cant18,cant19,cant20,cant21,cant22,cant23,cant24,cant25,cant26,cant27,cant28,cant29,cant30,cant31,cant32,cant33,cant34,cant35,cant36,cant37,cant38,cant39,cant40,cant41,cant42,cant43,cant44,cant45,cant46,cant47,cant48,cant49,cant50,cant51,cant52,cant53,cant54,cant55,cant56,cant57,cant58,cant59,cant60,cant61,cant62,cant63,cant64,cant65,cant66,cant67,cant68,cant69,cant70,cant71,cant72,cant73,cant74,cant75,cant76,cant77,cant78,cant79,cant80,cant81,cant82,cant83,cant84,cant85,cant86,cant87,cant88,cant89,cant90,cant91,cant92,cant93,cant94,cant95,cant96,cant97,cant98,cant99,cant100,cant101,cant102,cant103,cant104,cant105,cant106,cant107,cant108,cant109,cant110,cant111,cant112,cant113,cant114,cant115,cant116,cant117,cant118,cant119,cant120,cant121])
left_box0_1 = widgets.VBox([cant122,cant123,cant124,cant125,cant126,cant127,cant128,cant129,cant130,cant131,cant132,cant133,cant134,cant135,cant136,cant137,cant138,cant139,cant140,cant141,cant142,cant143,cant144,cant145,cant146,cant147,cant148,cant149,cant150,cant151,cant152,cant153,cant154,cant155,cant156,cant157,cant158,cant159,cant160,cant161,cant162,cant163,cant164,cant165,cant166,cant167,cant168,cant169,cant170,cant171,cant172,cant173,cant174,cant175,cant176,cant177,cant178,cant179,cant180,cant181,cant182,cant183,cant184,cant185,cant186,cant187,cant188,cant189,cant190,cant191,cant192,cant193,cant194,cant195,cant196,cant197,cant198,cant199,cant200,cant201,cant202,cant203,cant204,cant205,cant206,cant207,cant208,cant209,cant210,cant211,cant212,cant213,cant214,cant215,cant216,cant217,cant218,cant219,cant220,cant221,cant222,cant223,cant224,cant225,cant226,cant227,cant228,cant229,cant230,cant231,cant232,cant233,cant234,cant235,cant236,cant237,cant238,cant239,cant240,cant241,cant242])
left_box0 = widgets.HBox([left_box0_0,left_box0_1])
tab0 = widgets.HBox([left_box0, right_box])

left_box1 = widgets.VBox([a0,Es,q,RH])
tab1 = widgets.HBox([left_box1, right_box])

left_box2 = widgets.VBox([h,w,d,v,c,l])
tab2 = widgets.HBox([left_box2, right_box])

left_box3 = widgets.VBox([ham,epsilon1,eps,eps0,sigmat,sigmas,debye,eta,epsilon2,ljmin,lenght])
tab3 = widgets.HBox([left_box3, right_box])

left_box4 = widgets.VBox([nper,npp,nperfin,zcmin,dzc,fixzc])
tab4 = widgets.HBox([left_box4, right_box])

left_box5 = widgets.VBox([hilos,run])
tab5 = widgets.HBox([left_box5, right_box])
left_box5.children[1].on_click(inputs_processor)

tab6 = widgets.VBox([dbb])
tab6.children[0].on_click(databasef)

tabset=widgets.Tab([tab0, tab1, tab2 ,tab3, tab4, tab5, tab6])


tabset.set_title(0, 'Cantilevers')
tabset.set_title(1, 'Variables')
tabset.set_title(2, 'Forces')
tabset.set_title(3, 'Parameters')
tabset.set_title(4, 'Numerical Integration Values')
tabset.set_title(5, 'Execute')
tabset.set_title(6, 'Data Base')

display(tabset)


In [ ]:
#@title
UBI=os.getcwd()

def qfa (x=False):
    if x:
        #=============GRAPHICAL OR QUANTITATIVE#########
        question2 = widgets.HTML(value='Analysis options: ', style=style, layout=layout)
        graf = widgets.Button(value=False,description='Graphical',disabled=False, style=style, layout={'width': '300px'})
        quan = widgets.Button(value=False,description='Quantitative',disabled=False, style=style, layout={'width': '300px'})

        display(question2,graf,quan)
        graf.on_click(graficador)
        quan.on_click(cuantitativo)

def graficador (x=False):
    if x:
        directory = os.getcwd()
        os.chdir(directory+'/EXECUTE_pyDAMPF')
        if not os.path.exists("PLOTS"):
            os.mkdir("PLOTS")
        mascara=[cant1.value,cant2.value,cant3.value,cant4.value,cant5.value,cant6.value,cant7.value,cant8.value,cant9.value,cant10.value
                 ,cant11.value,cant12.value,cant13.value,cant14.value,cant15.value,cant16.value,cant17.value,cant18.value,cant19.value,cant20.value
                 ,cant21.value,cant22.value,cant23.value,cant24.value,cant25.value,cant26.value,cant27.value,cant28.value,cant29.value,cant30.value
                 ,cant31.value,cant32.value,cant33.value,cant34.value,cant35.value,cant36.value,cant37.value,cant38.value,cant39.value,cant40.value
                 ,cant41.value,cant42.value,cant43.value,cant44.value,cant45.value,cant46.value,cant47.value,cant48.value,cant49.value,cant50.value
                 ,cant51.value,cant52.value,cant53.value,cant54.value,cant55.value,cant56.value,cant57.value,cant58.value,cant59.value,cant60.value
                 ,cant61.value,cant62.value,cant63.value,cant64.value,cant65.value,cant66.value,cant67.value,cant68.value,cant69.value,cant70.value
                 ,cant71.value,cant72.value,cant73.value,cant74.value,cant75.value,cant76.value,cant77.value,cant78.value,cant79.value,cant80.value
                 ,cant81.value,cant82.value,cant83.value,cant84.value,cant85.value,cant86.value,cant87.value,cant88.value,cant89.value,cant90.value
                 ,cant91.value,cant92.value,cant93.value,cant94.value,cant95.value,cant96.value,cant97.value,cant98.value,cant99.value,cant100.value
                 ,cant101.value,cant102.value,cant103.value,cant104.value,cant105.value,cant106.value,cant107.value,cant108.value,cant109.value,cant110.value
                 ,cant111.value,cant112.value,cant113.value,cant114.value,cant115.value,cant116.value,cant117.value,cant118.value,cant119.value,cant120.value
                 ,cant121.value,cant122.value,cant123.value,cant124.value,cant125.value,cant126.value,cant127.value,cant128.value,cant129.value,cant130.value
                 ,cant131.value,cant132.value,cant133.value,cant134.value,cant135.value,cant136.value,cant137.value,cant138.value,cant139.value,cant140.value
                 ,cant141.value,cant142.value,cant143.value,cant144.value,cant145.value,cant146.value,cant147.value,cant148.value,cant149.value,cant150.value
                 ,cant151.value,cant152.value,cant153.value,cant154.value,cant155.value,cant156.value,cant157.value,cant158.value,cant159.value,cant160.value
                 ,cant161.value,cant162.value,cant163.value,cant164.value,cant165.value,cant166.value,cant167.value,cant168.value,cant169.value,cant170.value
                 ,cant171.value,cant172.value,cant173.value,cant174.value,cant175.value,cant176.value,cant177.value,cant178.value,cant179.value,cant180.value
                 ,cant181.value,cant182.value,cant183.value,cant184.value,cant185.value,cant186.value,cant187.value,cant188.value,cant189.value,cant190.value
                 ,cant191.value,cant192.value,cant193.value,cant194.value,cant195.value,cant196.value,cant197.value,cant198.value,cant199.value,cant200.value
                 ,cant201.value,cant202.value,cant203.value,cant204.value,cant205.value,cant206.value,cant207.value,cant208.value,cant209.value,cant210.value
                 ,cant211.value,cant212.value,cant213.value,cant214.value,cant215.value,cant216.value,cant217.value,cant218.value,cant219.value,cant220.value
                 ,cant221.value,cant222.value,cant223.value,cant224.value,cant225.value,cant226.value,cant227.value,cant228.value,cant229.value,cant230.value
                 ,cant231.value,cant232.value,cant233.value,cant234.value,cant235.value,cant236.value,cant237.value,cant238.value,cant239.value,cant240.value
                 ,cant241.value,cant242.value]
        f_name = "mascara_aux.txt"
        np.savetxt(f_name, mascara)
        #exec(open("Graphical_analysis.py").read())
        %run 'Graphical_analysis.ipynb'
        os.chdir(UBI)

def cuantitativo (x=False):
    if x:
        directory = os.getcwd()
        os.chdir(directory+'/EXECUTE_pyDAMPF')
        model = pd.read_csv("cantileversdata_model.csv")
        mascara=[cant1.value,cant2.value,cant3.value,cant4.value,cant5.value,cant6.value,cant7.value,cant8.value,cant9.value,cant10.value
                 ,cant11.value,cant12.value,cant13.value,cant14.value,cant15.value,cant16.value,cant17.value,cant18.value,cant19.value,cant20.value
                 ,cant21.value,cant22.value,cant23.value,cant24.value,cant25.value,cant26.value,cant27.value,cant28.value,cant29.value,cant30.value
                 ,cant31.value,cant32.value,cant33.value,cant34.value,cant35.value,cant36.value,cant37.value,cant38.value,cant39.value,cant40.value
                 ,cant41.value,cant42.value,cant43.value,cant44.value,cant45.value,cant46.value,cant47.value,cant48.value,cant49.value,cant50.value
                 ,cant51.value,cant52.value,cant53.value,cant54.value,cant55.value,cant56.value,cant57.value,cant58.value,cant59.value,cant60.value
                 ,cant61.value,cant62.value,cant63.value,cant64.value,cant65.value,cant66.value,cant67.value,cant68.value,cant69.value,cant70.value
                 ,cant71.value,cant72.value,cant73.value,cant74.value,cant75.value,cant76.value,cant77.value,cant78.value,cant79.value,cant80.value
                 ,cant81.value,cant82.value,cant83.value,cant84.value,cant85.value,cant86.value,cant87.value,cant88.value,cant89.value,cant90.value
                 ,cant91.value,cant92.value,cant93.value,cant94.value,cant95.value,cant96.value,cant97.value,cant98.value,cant99.value,cant100.value
                 ,cant101.value,cant102.value,cant103.value,cant104.value,cant105.value,cant106.value,cant107.value,cant108.value,cant109.value,cant110.value
                 ,cant111.value,cant112.value,cant113.value,cant114.value,cant115.value,cant116.value,cant117.value,cant118.value,cant119.value,cant120.value
                 ,cant121.value,cant122.value,cant123.value,cant124.value,cant125.value,cant126.value,cant127.value,cant128.value,cant129.value,cant130.value
                 ,cant131.value,cant132.value,cant133.value,cant134.value,cant135.value,cant136.value,cant137.value,cant138.value,cant139.value,cant140.value
                 ,cant141.value,cant142.value,cant143.value,cant144.value,cant145.value,cant146.value,cant147.value,cant148.value,cant149.value,cant150.value
                 ,cant151.value,cant152.value,cant153.value,cant154.value,cant155.value,cant156.value,cant157.value,cant158.value,cant159.value,cant160.value
                 ,cant161.value,cant162.value,cant163.value,cant164.value,cant165.value,cant166.value,cant167.value,cant168.value,cant169.value,cant170.value
                 ,cant171.value,cant172.value,cant173.value,cant174.value,cant175.value,cant176.value,cant177.value,cant178.value,cant179.value,cant180.value
                 ,cant181.value,cant182.value,cant183.value,cant184.value,cant185.value,cant186.value,cant187.value,cant188.value,cant189.value,cant190.value
                 ,cant191.value,cant192.value,cant193.value,cant194.value,cant195.value,cant196.value,cant197.value,cant198.value,cant199.value,cant200.value
                 ,cant201.value,cant202.value,cant203.value,cant204.value,cant205.value,cant206.value,cant207.value,cant208.value,cant209.value,cant210.value
                 ,cant211.value,cant212.value,cant213.value,cant214.value,cant215.value,cant216.value,cant217.value,cant218.value,cant219.value,cant220.value
                 ,cant221.value,cant222.value,cant223.value,cant224.value,cant225.value,cant226.value,cant227.value,cant228.value,cant229.value,cant230.value
                 ,cant231.value,cant232.value,cant233.value,cant234.value,cant235.value,cant236.value,cant237.value,cant238.value,cant239.value,cant240.value
                 ,cant241.value,cant242.value]
        modelaux = model[mascara]
        modelaux .to_csv("cantileversdata_modelaux.csv")
        %run 'Quantitative_analysis.ipynb'
        os.chdir(UBI)


question = widgets.HTML(value='The simulation was completed?  :', style=style, layout=layout)
yes = widgets.Button(value=False,description='Yes',disabled=False, style=style, layout={'width': '300px'})

display(question,yes)
yes.on_click(qfa)

# Previous Results

In [ ]:
#@title Previous results
UBI=os.getcwd()
directory = None

EXECUTIONS_DIR = "/home/jovyan/executions"
os.makedirs(EXECUTIONS_DIR, exist_ok=True)
results = os.listdir(EXECUTIONS_DIR)
results.sort(reverse=True)
dropdown = widgets.Dropdown(
    options=results,
    description='Last results:',
    disabled=False,
)

def qfa_gallery (x=False):
    if x:
        #=============GRAPHICAL OR QUANTITATIVE#########
        question2 = widgets.HTML(value='Analysis options: ', style=style, layout=layout)
        graf = widgets.Button(value=False,description='Graphical',disabled=False, style=style, layout={'width': '300px'})
        quan = widgets.Button(value=False,description='Quantitative',disabled=False, style=style, layout={'width': '300px'})

        graf.directory = os.path.join(EXECUTIONS_DIR, dropdown.value)
        quan.directory = os.path.join(EXECUTIONS_DIR, dropdown.value)

        display(question2,graf,quan)
        graf.on_click(graficador_gallery)
        quan.on_click(cuantitativo_gallery)

def graficador_gallery (x=False):
    if x:
        directory = x.directory
        os.chdir(directory)
        if not os.path.exists("PLOTS"):
            os.mkdir("PLOTS")
        mascara=[cant1.value,cant2.value,cant3.value,cant4.value,cant5.value,cant6.value,cant7.value,cant8.value,cant9.value,cant10.value
                 ,cant11.value,cant12.value,cant13.value,cant14.value,cant15.value,cant16.value,cant17.value,cant18.value,cant19.value,cant20.value
                 ,cant21.value,cant22.value,cant23.value,cant24.value,cant25.value,cant26.value,cant27.value,cant28.value,cant29.value,cant30.value
                 ,cant31.value,cant32.value,cant33.value,cant34.value,cant35.value,cant36.value,cant37.value,cant38.value,cant39.value,cant40.value
                 ,cant41.value,cant42.value,cant43.value,cant44.value,cant45.value,cant46.value,cant47.value,cant48.value,cant49.value,cant50.value
                 ,cant51.value,cant52.value,cant53.value,cant54.value,cant55.value,cant56.value,cant57.value,cant58.value,cant59.value,cant60.value
                 ,cant61.value,cant62.value,cant63.value,cant64.value,cant65.value,cant66.value,cant67.value,cant68.value,cant69.value,cant70.value
                 ,cant71.value,cant72.value,cant73.value,cant74.value,cant75.value,cant76.value,cant77.value,cant78.value,cant79.value,cant80.value
                 ,cant81.value,cant82.value,cant83.value,cant84.value,cant85.value,cant86.value,cant87.value,cant88.value,cant89.value,cant90.value
                 ,cant91.value,cant92.value,cant93.value,cant94.value,cant95.value,cant96.value,cant97.value,cant98.value,cant99.value,cant100.value
                 ,cant101.value,cant102.value,cant103.value,cant104.value,cant105.value,cant106.value,cant107.value,cant108.value,cant109.value,cant110.value
                 ,cant111.value,cant112.value,cant113.value,cant114.value,cant115.value,cant116.value,cant117.value,cant118.value,cant119.value,cant120.value
                 ,cant121.value,cant122.value,cant123.value,cant124.value,cant125.value,cant126.value,cant127.value,cant128.value,cant129.value,cant130.value
                 ,cant131.value,cant132.value,cant133.value,cant134.value,cant135.value,cant136.value,cant137.value,cant138.value,cant139.value,cant140.value
                 ,cant141.value,cant142.value,cant143.value,cant144.value,cant145.value,cant146.value,cant147.value,cant148.value,cant149.value,cant150.value
                 ,cant151.value,cant152.value,cant153.value,cant154.value,cant155.value,cant156.value,cant157.value,cant158.value,cant159.value,cant160.value
                 ,cant161.value,cant162.value,cant163.value,cant164.value,cant165.value,cant166.value,cant167.value,cant168.value,cant169.value,cant170.value
                 ,cant171.value,cant172.value,cant173.value,cant174.value,cant175.value,cant176.value,cant177.value,cant178.value,cant179.value,cant180.value
                 ,cant181.value,cant182.value,cant183.value,cant184.value,cant185.value,cant186.value,cant187.value,cant188.value,cant189.value,cant190.value
                 ,cant191.value,cant192.value,cant193.value,cant194.value,cant195.value,cant196.value,cant197.value,cant198.value,cant199.value,cant200.value
                 ,cant201.value,cant202.value,cant203.value,cant204.value,cant205.value,cant206.value,cant207.value,cant208.value,cant209.value,cant210.value
                 ,cant211.value,cant212.value,cant213.value,cant214.value,cant215.value,cant216.value,cant217.value,cant218.value,cant219.value,cant220.value
                 ,cant221.value,cant222.value,cant223.value,cant224.value,cant225.value,cant226.value,cant227.value,cant228.value,cant229.value,cant230.value
                 ,cant231.value,cant232.value,cant233.value,cant234.value,cant235.value,cant236.value,cant237.value,cant238.value,cant239.value,cant240.value
                 ,cant241.value,cant242.value]
        f_name = "mascara_aux.txt"
        np.savetxt(f_name, mascara)
        #exec(open("Graphical_analysis.py").read())
        %run 'Graphical_analysis.ipynb'
        os.chdir(UBI)

def cuantitativo_gallery (x=False):
    if x:
        directory = x.directory
        os.chdir(directory)
        model = pd.read_csv("cantileversdata_model.csv")
        mascara=[cant1.value,cant2.value,cant3.value,cant4.value,cant5.value,cant6.value,cant7.value,cant8.value,cant9.value,cant10.value
                 ,cant11.value,cant12.value,cant13.value,cant14.value,cant15.value,cant16.value,cant17.value,cant18.value,cant19.value,cant20.value
                 ,cant21.value,cant22.value,cant23.value,cant24.value,cant25.value,cant26.value,cant27.value,cant28.value,cant29.value,cant30.value
                 ,cant31.value,cant32.value,cant33.value,cant34.value,cant35.value,cant36.value,cant37.value,cant38.value,cant39.value,cant40.value
                 ,cant41.value,cant42.value,cant43.value,cant44.value,cant45.value,cant46.value,cant47.value,cant48.value,cant49.value,cant50.value
                 ,cant51.value,cant52.value,cant53.value,cant54.value,cant55.value,cant56.value,cant57.value,cant58.value,cant59.value,cant60.value
                 ,cant61.value,cant62.value,cant63.value,cant64.value,cant65.value,cant66.value,cant67.value,cant68.value,cant69.value,cant70.value
                 ,cant71.value,cant72.value,cant73.value,cant74.value,cant75.value,cant76.value,cant77.value,cant78.value,cant79.value,cant80.value
                 ,cant81.value,cant82.value,cant83.value,cant84.value,cant85.value,cant86.value,cant87.value,cant88.value,cant89.value,cant90.value
                 ,cant91.value,cant92.value,cant93.value,cant94.value,cant95.value,cant96.value,cant97.value,cant98.value,cant99.value,cant100.value
                 ,cant101.value,cant102.value,cant103.value,cant104.value,cant105.value,cant106.value,cant107.value,cant108.value,cant109.value,cant110.value
                 ,cant111.value,cant112.value,cant113.value,cant114.value,cant115.value,cant116.value,cant117.value,cant118.value,cant119.value,cant120.value
                 ,cant121.value,cant122.value,cant123.value,cant124.value,cant125.value,cant126.value,cant127.value,cant128.value,cant129.value,cant130.value
                 ,cant131.value,cant132.value,cant133.value,cant134.value,cant135.value,cant136.value,cant137.value,cant138.value,cant139.value,cant140.value
                 ,cant141.value,cant142.value,cant143.value,cant144.value,cant145.value,cant146.value,cant147.value,cant148.value,cant149.value,cant150.value
                 ,cant151.value,cant152.value,cant153.value,cant154.value,cant155.value,cant156.value,cant157.value,cant158.value,cant159.value,cant160.value
                 ,cant161.value,cant162.value,cant163.value,cant164.value,cant165.value,cant166.value,cant167.value,cant168.value,cant169.value,cant170.value
                 ,cant171.value,cant172.value,cant173.value,cant174.value,cant175.value,cant176.value,cant177.value,cant178.value,cant179.value,cant180.value
                 ,cant181.value,cant182.value,cant183.value,cant184.value,cant185.value,cant186.value,cant187.value,cant188.value,cant189.value,cant190.value
                 ,cant191.value,cant192.value,cant193.value,cant194.value,cant195.value,cant196.value,cant197.value,cant198.value,cant199.value,cant200.value
                 ,cant201.value,cant202.value,cant203.value,cant204.value,cant205.value,cant206.value,cant207.value,cant208.value,cant209.value,cant210.value
                 ,cant211.value,cant212.value,cant213.value,cant214.value,cant215.value,cant216.value,cant217.value,cant218.value,cant219.value,cant220.value
                 ,cant221.value,cant222.value,cant223.value,cant224.value,cant225.value,cant226.value,cant227.value,cant228.value,cant229.value,cant230.value
                 ,cant231.value,cant232.value,cant233.value,cant234.value,cant235.value,cant236.value,cant237.value,cant238.value,cant239.value,cant240.value
                 ,cant241.value,cant242.value]
        modelaux = model[mascara]
        modelaux .to_csv("cantileversdata_modelaux.csv")
        %run 'Quantitative_analysis.ipynb'
        os.chdir(UBI)



use_result = widgets.Button(value=False,description='Use this result',disabled=False, style=style, layout={'width': '300px'})


use_result.on_click(qfa_gallery)



display(dropdown)
display(use_result)